In [ ]:
%run GpuOptions.ipynb
%run BuildTrainingData.ipynb

In [11]:
from common import frozenmodel
from common.classifier import Classifier

# use the best weights so far so that we can test the model as it is being trained
model = frozenmodel.generate(weights_filepath='isthemountainout.best.h5')
classifier = Classifier(model=model, labels=frozenmodel.labels())

In [12]:
import glob
import os
import random
from PIL import Image

for classname in os.listdir('TrainingData'):
    print('--------------------------------------------------------')
    print('Quick Test: {classname}'.format(classname=classname))
    print('{classification:<20} {confidence:<20} {filepath}'.format(
        classification='Classification',
        confidence='Confidence',
        filepath='File Path'))
    print('--------------------------------------------------------')
    for filepath in random.choices(list(glob.iglob(f'TrainingData/{classname}/*.jpg')), k=3):
        image = Image.open(filepath)
        classification, confidence = classifier.classify(image)
        print('{classification:<20} {confidence:<20} {filepath}'.format(
            classification=classification,
            confidence=confidence,
            filepath=os.path.basename(filepath)))


--------------------------------------------------------
Quick Test: Beautiful
Classification       Confidence           File Path
--------------------------------------------------------
Label.MYSTICAL       84.70063805580139    MountRainier_2020-08-02T072000.jpg
Label.BEAUTIFUL      64.75191116333008    MountRainier_2020-12-04T155000.jpg
Label.MYSTICAL       56.691354513168335   offset_0_1380521001818095621-EyiYiPYVcAMFCYJ.jpg
--------------------------------------------------------
Quick Test: Mystical
Classification       Confidence           File Path
--------------------------------------------------------
Label.NOT_VISIBLE    56.10477924346924    MountRainier_2020-08-24T194000.jpg
Label.MYSTICAL       81.77843689918518    MountRainier_2020-09-06T140000.jpg
Label.MYSTICAL       96.72260880470276    MountRainier_2020-08-28T152000.jpg
--------------------------------------------------------
Quick Test: Night
Classification       Confidence           File Path
----------------------

In [ ]:
import numpy as np
import sklearn.metrics
import itertools
from matplotlib import pyplot as plt

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)\
                .flow_from_directory(data_directory, batch_size=4096, shuffle=True, target_size=image_size)

class_names = list(datagen.class_indices.keys())
data, labels = datagen.next()
test_pred_raw = model.predict(data)
test_pred = np.argmax(test_pred_raw, axis=1)
test_label = np.argmax(labels, axis=1)

def plot_confusion_matrix(cm, *, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
       color = "white" if cm[i, j] > threshold else "black"
       plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)
    plt.tight_layout()
    plt.subplots_adjust(left=0.125, right=0.9, bottom=0.1, top=0.9, wspace=0.2, hspace=0.2)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cm = sklearn.metrics.confusion_matrix(test_label, test_pred)
plot_confusion_matrix(cm, class_names=class_names)

Found 8367 images belonging to 4 classes.
